In [8]:
import os, sys
import numpy as np
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
from astropy import units
from tqdm import tqdm_notebook
from lightkurve.lightcurve import LightCurve
from scipy.interpolate import interp1d

from costools import timefilter, splittag
import calcos

%load_ext autoreload
%autoreload 2
sys.path.append('/Users/arcticfox/Documents/GitHub/cos_flares/src')
from cos_reduction import *

## PROPOSAL ID 16164
plt.rcParams['font.size']=24

rootname = ['lebb01s7q', 'lebb01sbq', 'lebb01sdq', 'lebb01siq', 'lebb01smq']
rootname = ['lebb03xpq', 'lebb03xrq', 'lebb03xtq', 'lebb03xvq', 'lebb03xxq']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
%load_ext autoreload
%autoreload 2
sys.path.append('/Users/arcticfox/Documents/GitHub/cos_flares/src')
from cos_reduction import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
cr = cosReduce(rootname=rootname, 
               input_path='/Users/arcticfox/Documents/AUMic/rawfiles/')

In [11]:
cr.split_corrtag(output_path='/Users/arcticfox/Documents/AUMic/splittag2')

/Users/arcticfox/Documents/AUMic/rawfiles/lebb03xpq_corrtag_a.fits
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_1_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_2_corrtag_a.fits written


/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_3_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_4_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_5_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_6_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_7_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_8_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_9_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_10_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_11_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_12_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_13_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_14_corrtag_a.

/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_30_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_31_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_32_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_33_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_34_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_35_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_36_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_37_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_38_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_39_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_40_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xpq_41_cor

/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_56_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_57_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_58_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_59_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_60_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_61_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_62_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_63_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_64_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_65_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_66_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_67_cor

/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_70_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_71_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_72_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_73_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_74_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_75_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_76_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_77_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_78_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_79_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_80_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xrq_81_cor

/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xtq_83_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xtq_1_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xtq_2_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xtq_3_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xtq_4_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xtq_5_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xtq_6_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xtq_7_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xtq_8_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xtq_9_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xtq_10_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xtq_11_corrtag_b.fi

/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_13_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_14_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_15_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_16_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_17_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_18_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_19_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_20_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_21_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_22_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_23_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_24_cor

/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_27_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_28_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_29_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_30_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_31_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_32_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_33_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_34_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_35_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_36_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_37_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xvq_38_cor

/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_40_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_41_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_42_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_43_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_44_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_45_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_46_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_47_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_48_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_49_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_50_corrtag_a.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_51_cor

/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_54_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_55_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_56_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_57_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_58_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_59_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_60_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_61_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_62_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_63_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_64_corrtag_b.fits written
/Users/arcticfox/Documents/AUMic/splittag2/split_lebb03xxq_65_cor

In [25]:
cr.check_ref_files(path='/Users/arcticfox/Documents/AUMic/lref/')

'You have all reference files needed!'

In [ ]:
cr.reduce_data(output_path='/Users/arcticfox/Documents/AUMic/reduced2')

  0%|          | 0/402 [00:00<?, ?it/s]

CALCOS version 3.3.11
numpy version 1.21.2
astropy version 4.2.1
Begin 24-Sep-2021 12:51:00 EDT
Shifting to 190, -3
Shifting to 192, -3
Shifting to 190, -2
Shifting to 192, -2
Shifting to 190, -1
Shifting to 192, -1
Shifting to 190, 0
Shifting to 192, 0
Shifting to 190, 1
Shifting to 192, 1
Shifting to 190, 2
Shifting to 192, 2
Shifting to 190, 3
Shifting to 192, 3
Shifting to 190, 4
Shifting to 192, 4
Shifting to 190, 5
Shifting to 192, 5
Shifting to 190, 6
Shifting to 192, 6
Extraction algorithm = TWOZONE


Two-zone extraction method
Using profile centroid of 423.548680
Calculating background for reference profile
Using extraction percentiles of:
0.500000 and 99.500000 (outer region)
and 10.000000 and 90.000000 (inner region)
Shifting to 189, 0
Shifting to 190, 0
Shifting to 191, 0
Shifting to 192, 0
Shifting to 193, 0
Shifting to 189, 1
Shifting to 190, 1
Shifting to 191, 1
Shifting to 192, 1
Shifting to 193, 1
Shifting to 189, 2
Shifting to 190, 2
Shifting to 191, 2
Shifting to 192, 2
Shifting to 193, 2
Shifting to 189, 3
Shifting to 190, 3
Shifting to 191, 3
Shifting to 192, 3
Shifting to 193, 3
Shifting to 189, 4
Shifting to 190, 4
Shifting to 191, 4
Shifting to 192, 4
Shifting to 193, 4
Extraction algorithm = TWOZONE
Two-zone extraction method
Using profile centroid of 485.093382
Calculating background for reference profile
Using extraction percentiles of:
0.500000 and 99.500000 (outer region)
and 10.000000 and 90.000000 (inner region)


  0%|          | 1/402 [00:24<2:44:52, 24.67s/it]

CALCOS version 3.3.11
numpy version 1.21.2
astropy version 4.2.1
Shifting to 190, -3
Shifting to 192, -3
Shifting to 190, -2
Shifting to 192, -2
Shifting to 190, -1
Shifting to 192, -1
Shifting to 190, 0
Shifting to 192, 0
Shifting to 190, 1
Shifting to 192, 1
Shifting to 190, 2
Shifting to 192, 2
Shifting to 190, 3
Shifting to 192, 3
Shifting to 190, 4
Shifting to 192, 4
Shifting to 190, 5
Shifting to 192, 5
Extraction algorithm = TWOZONE
Two-zone extraction method
Using profile centroid of 423.548680
Calculating background for reference profile
Using extraction percentiles of:
0.500000 and 99.500000 (outer region)
and 10.000000 and 90.000000 (inner region)
Shifting to 189, 0
Shifting to 190, 0
Shifting to 191, 0
Shifting to 192, 0
Shifting to 189, 1
Shifting to 190, 1
Shifting to 191, 1
Shifting to 192, 1
Shifting to 189, 2
Shifting to 190, 2
Shifting to 191, 2
Shifting to 192, 2
Shifting to 189, 3
Shifting to 190, 3
Shifting to 191, 3
Shifting to 192, 3
Shifting to 189, 4
Shifting t

...

CALCOS version 3.3.11
numpy version 1.21.2
astropy version 4.2.1
Shifting to 188, -3
Shifting to 190, -3
Shifting to 191, -3
Shifting to 188, -2
Shifting to 190, -2
Shifting to 191, -2
Shifting to 188, -1
Shifting to 190, -1
Shifting to 191, -1
Shifting to 188, 0
Shifting to 190, 0
Shifting to 191, 0
Shifting to 188, 1
Shifting to 190, 1
Shifting to 191, 1
Shifting to 188, 2
Shifting to 190, 2
Shifting to 191, 2
Shifting to 188, 3
Shifting to 190, 3
Shifting to 191, 3
Shifting to 188, 4
Shifting to 190, 4
Shifting to 191, 4
Shifting to 188, 5
Shifting to 190, 5
Shifting to 191, 5
Shifting to 188, 6
Shifting to 190, 6
Shifting to 191, 6
Extraction algorithm = TWOZONE
Two-zone extraction method
Using profile centroid of 423.548575
Calculating background for reference profile
Using extraction percentiles of:
0.500000 and 99.500000 (outer region)
and 10.000000 and 90.000000 (inner region)
Shifting to 188, -1
Shifting to 190, -1
Shifting to 191, -1
Shifting to 188, 0
Shifting to 190, 0
Shif

 31%|███       | 125/402 [51:24<1:53:34, 24.60s/it]

CALCOS version 3.3.11
numpy version 1.21.2
astropy version 4.2.1
Shifting to 188, -3
Shifting to 190, -3
Shifting to 191, -3
Shifting to 188, -2
Shifting to 190, -2
Shifting to 191, -2
Shifting to 188, -1
Shifting to 190, -1
Shifting to 191, -1
Shifting to 188, 0
Shifting to 190, 0
Shifting to 191, 0
Shifting to 188, 1
Shifting to 190, 1
Shifting to 191, 1
Shifting to 188, 2
Shifting to 190, 2
Shifting to 191, 2
Shifting to 188, 3
Shifting to 190, 3
Shifting to 191, 3
Shifting to 188, 4
Shifting to 190, 4
Shifting to 191, 4
Shifting to 188, 5
Shifting to 190, 5
Shifting to 191, 5
Shifting to 188, 6
Shifting to 190, 6
Shifting to 191, 6
Extraction algorithm = TWOZONE
Two-zone extraction method
Using profile centroid of 423.548575
Calculating background for reference profile
Using extraction percentiles of:
0.500000 and 99.500000 (outer region)
and 10.000000 and 90.000000 (inner region)
Shifting to 189, 0
Shifting to 191, 0
Shifting to 192, 0
Shifting to 189, 1
Shifting to 191, 1
Shiftin

 31%|███▏      | 126/402 [51:49<1:54:02, 24.79s/it]

CALCOS version 3.3.11
numpy version 1.21.2
astropy version 4.2.1
Shifting to 188, -3
Shifting to 190, -3
Shifting to 191, -3
Shifting to 188, -2
Shifting to 190, -2
Shifting to 191, -2
Shifting to 188, -1
Shifting to 190, -1
Shifting to 191, -1
Shifting to 188, 0
Shifting to 190, 0
Shifting to 191, 0
Shifting to 188, 1
Shifting to 190, 1
Shifting to 191, 1
Shifting to 188, 2
Shifting to 190, 2
Shifting to 191, 2
Shifting to 188, 3
Shifting to 190, 3
Shifting to 191, 3
Shifting to 188, 4
Shifting to 190, 4
Shifting to 191, 4
Shifting to 188, 5
Shifting to 190, 5
Shifting to 191, 5
Shifting to 188, 6
Shifting to 190, 6
Shifting to 191, 6
Extraction algorithm = TWOZONE
Two-zone extraction method
Using profile centroid of 423.548575
Calculating background for reference profile
Using extraction percentiles of:
0.500000 and 99.500000 (outer region)
and 10.000000 and 90.000000 (inner region)
Shifting to 189, 0
Shifting to 191, 0
Shifting to 192, 0
Shifting to 189, 1
Shifting to 191, 1
Shiftin

## Plotting x1d files

In [ ]:
adir = './split_output_a'
bdir = './split_output_b'
fn_a = np.sort([os.path.join(adir, i) for i in os.listdir(adir) if i.endswith('x1d.fits')])
fn_b = np.sort([os.path.join(bdir, i) for i in os.listdir(bdir) if i.endswith('x1d.fits')])

In [ ]:
## FIXES POOR NAMING CONVENTION
new = []
cp = []
deflen = len('./split_output_b/split_lebb01sdq_70_x1d.fits')


for fn in fn_b:
    if len(fn) < deflen:
        cp.append(fn)
        r = fn.split('_')
        r[-2] = '0'+r[-2]
        new.append('_'.join(e for e in r))
        
for i in range(len(new)):
    os.replace(cp[i], new[i])

In [ ]:
orbits = np.zeros(len(fn_b), dtype=int)

for i,fn in enumerate(fn_a):
    r = fn.split('_')[-3]
    orbits[i] = np.where(np.array(rootname)==r)[0]

In [ ]:
which = fn_b

parula = np.load('/Users/arcticfox/parula_colors.npy')
parula = parula[np.linspace(0,len(parula)-50,len(rootname),1,dtype=int)]

In [ ]:
%matplotlib inline
plt.figure(figsize=(14,4))

for i in range(len(fn_a)):
    x1ddata = fits.getdata(fn_a[i])

    #wavelengtha = np.concatenate((x1ddata["wavelength"][1],
    #                            x1ddata["wavelength"][0]))
    #fluxa = np.concatenate((x1ddata["flux"][1], 
    #                       x1ddata["flux"][0]))
    #erra = np.concatenate((x1ddata["error"][1],
    #                      x1ddata["error"][0]))
    
    x1ddata = fits.getdata(fn_b[i])
    wavelengthb = np.concatenate((x1ddata["wavelength"][1],
                                x1ddata["wavelength"][0]))
    fluxb = np.concatenate((x1ddata["flux"][1], 
                           x1ddata["flux"][0]))
    errb = np.concatenate((x1ddata["error"][1], 
                           x1ddata["error"][0]))
    
    if i == 0:
        all_wave = np.zeros((len(fn_b), len(wavelengthb)))
        all_flux = np.zeros((len(fn_b), len(fluxb)))
        all_err  = np.zeros((len(fn_b), len(errb)))
        
    
    
    #plt.plot(wavelength, fluxa+fluxb)
    
    all_wave[i] = wavelengthb#np.nanmedian([wavelengtha,wavelengthb],axis=0) + 0.0
    all_flux[i] = fluxb #+ fluxb
    all_err[i] = errb

In [ ]:
all_wave[0]

### interpolated onto same grid

In [ ]:
length = int(all_wave.shape[1]*1.5)
interp_flux = np.zeros((len(fn_a), length-2))
interp_err = np.zeros((len(fn_a), length-2))

finer_wavelength = np.flip(np.logspace(np.log10(np.nanmin(all_wave[:,-1])), 
                                       np.log10(np.nanmax(all_wave[:,0])),
                                       length, base=10.0))[1:-1]

for i in range(len(all_flux)):
    f = interp1d(all_wave[i], all_flux[i])
    interp_flux[i] = f(finer_wavelength)+0.0
    f = interp1d(all_wave[i], all_err[i])
    interp_err[i] = f(finer_wavelength)+0.0

In [ ]:
binsize=3
binned_flux = np.zeros((interp_flux.shape[0], 18725))
binned_err = np.zeros((interp_flux.shape[0], 18725))
for i in tqdm_notebook(range(len(interp_flux))):
    lk = LightCurve(time=finer_wavelength, 
                    flux=interp_flux[i], flux_err=interp_err[i]).bin(binsize=binsize)
    binned_flux[i] = lk.flux.value
    binned_err[i] = lk.flux_err.value
binned_wavelenth = lk.time.value + 0.0

In [ ]:
%matplotlib inline

plt.figure(figsize=(14,4))
i = 10
mask = interp_flux[i] > 0
lk = LightCurve(time=finer_wavelength[mask],
                flux=interp_flux[i][mask],
                flux_err=interp_err[i][mask]).bin(binsize=binsize)
plt.errorbar(finer_wavelength[mask],
             interp_flux[i][mask],
             yerr=interp_err[i][mask],
             marker='o', linestyle='', color='k')
plt.errorbar(lk.time.value,
             lk.flux.value,
             yerr=lk.flux_err.value,
             marker='o', linestyle='')
plt.ylim(0,5e-13)
plt.xlim(1175.5,1176)

In [ ]:
times = np.zeros((len(fn_a),2))
for i in range(len(fn_a)):
    hdu = fits.open(fn_a[i])
    start = ((hdu[1].header['EXPSTART'] - 59362.14828)*units.day).to(units.s)
    t = [float(i) for i in str(hdu[0].header['HISTORY']).split('was')[-1][1:].split(' to ')]
    times[i][0] = t[0] + start.value
    times[i][1] = t[1] + start.value
    hdu.close()
times = np.nanmedian(times, axis=1)

In [ ]:
from astropy.table import Table

In [ ]:
tab = Table()
tab['time_s'] = times
for key in list(widths.keys()):
    tab['ew_'+key] = widths[key]
tab['orbits'] = orbits
tab.write('ew.tab', format='ascii')

In [ ]:
np.save('data.npy', [finer_wavelength,
                     interp_flux, 
                     interp_err])
np.save('binned_data.npy', [binned_wavelenth,
                            binned_flux,
                            binned_err])

In [ ]:
np.save('times.npy', times)
np.save('orbits.npy', orbits)